# Perfect foresight experiments with dolo

** Spencer Lyon **

** Pablo Winant **

January 2018

In [1]:
#If running locally, you may need to install these packages, and if you have already installed them, then do a Pkg.update() to ensure you are using the latest version of Dolo

# Pkg.add("Dolo")
# Pkg.add("PlotlyJS")
# Pkg.add("AxisArrays")

In this notebook we will explore the effects of techonology and fiscal policy shocks in a nonstochastic model. The model and figures are inspired by chapter 11 of Recursive Macroeconomic Theory 3rd edition (RMT3) by Ljungqvist and Sargent. 

We will focus on the computational aspects of the exercises and will refer the interested reader to section 11.9 of RMT3 for a discussion on the underlying economics.

## Model

A representative household has preferences over consumption that are ordered by 

$$U := \sum_{t=0}^{\infty} \beta^t \frac{c_t^{1-\gamma}}{1-\gamma},$$

where $\beta = 0.95$ is the agent's discount factor and $\gamma$ is the coeffiicent of relative risk aversion.

A perfectly competitive representative firm rents capital $k_t$ from the household and produces using the production function $f(k) = A k^{\alpha}$. The aggregate technology in the economy is

$$g_t + c_t + k_{t+1} \le Ak_t^{\alpha} + (1-\delta) k_t,$$

where $g_t$ is government spending, $A$ is a constant productivity factor, and $\delta$ is the depreciation rate of capital.

The consumer faces two taxes: a consumption tax $\tau_{ct}$ and tax on earnings to capital $\tau_{kt}$. The household budget constraint is

$$ \sum_{t=0}^{\infty} q_t \left\{(1 + \tau_{ct}) c_t + [k_{t+1} - (1 - \delta) k_t] \right\} \le \sum_{t=0}^{\infty} q_t \left\{\eta_t k_t -\tau_{kt} (\eta_t - \delta) k_t + w_t\right\}.$$

Here $q_t$ is the time zero price of one unit of consumption in period $t$, $\eta_t$ is the pre-tax price households recieve by lending capital to firms in time $t$, and $w_t$ is the time $t$ wage households earn on the inelastically supplied one unit of labor.

The government faces the budget constraint

$$ \sum_{t=0}^{\infty} q_t g_t \le \sum_{t=0}^{\infty} q_t \left\{\tau_{ct} c_t + \tau_{kt} (\eta_t - \delta) k_t \right\}$$

## Equilibrium conditions

A competitive equilibrium is 

- a budget-feasible government policy $\left\{g_t, \tau_{ct}, \tau_{kt} \right\}_{t=0}^{\infty}$
- a feasible allocation $\left\{c_t, k_{t+1}\right\}_{t=0}^{\infty}$
- and prices $\left\{q_t \right\}_{t=0}^{\infty}$ 

such that

- given prices and the government policy, the allocation solves the household and firm problems.

### Firm optimization

The firm's problem is very simple:

$$\max_{k_t} \sum_{t=0}^{\infty} q_t \left[A k_t^{\alpha} - \eta_t k_t \right].$$

The zero-profit condition is $\eta_t = A \alpha k_t^{\alpha-1}$.

### Household optimization

The problem of the household is to maximize $U$ subject to the budget constraint, taking prices and the government policy as given.

Assuming an interior solution and that the budget constraint holds with equality, the first order necessary condition of the household can be summarized by an Euler equation

$$1 = \beta \left(\frac{c_{t+1}}{c_t}\right)^{-\gamma} \frac{(1+\tau_{ct})}{(1+\tau_{ct+1})} \left[(1 - \tau_{kt+1})(\alpha A k_{t+1}^{\alpha-1} - \delta) + 1 \right]$$

and a law of motion for capital

$$k_{t+1} = A k_{t}^{\alpha} + (1 - \delta) k_t - g_t - c_t.$$

### Prices and government policy

Imposing a no arbitrage condition on the household allows one to derive the following condition on prices (see RMT3 section 11.5.1 for more details):

$$\frac{q_t}{q_{t+1}} = \left[(1 - \tau_{kt+1}(\eta_{t+1} - \delta) + 1 \right].$$

After a normalization that $q_0 = 1$, the above equation pins down the sequence of prices.

In the experiments below we will assume that the goverment policy is exogenously given and solve for how the solutions to the household and firms problems adjust to shocks to the government policy.

### Other equilibrium conditions

There are a few other variables of interest. 
$$\begin{align*}
\eta_t &= \alpha A k_t^{\alpha-1} \\
w_t &= A k_t^{\alpha} - k_t \eta_t \\
\bar{R}_{t+1} &= \frac{(1+\tau_{ct})}{(1+\tau_{ct+1})} \left[(1 - \tau_{kt+1})(\alpha A k_{t+1}^{\alpha-1} - \delta) + 1 \right]
\end{align*}$$

Here $w_t$ is the wage rate the firm must pay to the household. Above we wrote the firm's problem with capital as the only input into production. All the equilibrium conditions above are still correct if we instead assume that the firm operates a constant returns to technology $F(k, n)$ and assume that the household inelastically supplies a single unit of labor and is paid a wage of $w_t$.

$\bar{R}_{t+1}$ is rate at which the price and taxes allow the conusmer to substitute consumption in period $t$ for consumption in period $t+1$.

### Experiments

We will do a number of experiments and analyze the transition path for the equilibrium in each case:

1. A foreseen once-and-for-all increase in $g$ from 0.2 to 0.4 in period 10.
2. A foreseen once-and-for-all increase in $\tau_c$ from 0.0 to 0.2 in period 10.
3. A foreseen once-and-for-all increase in $\tau_k$ from 0.0 to 0.2 in period 10.
3. A foreseen one-time increase in $g$ from 0.2 to 0.4 in period 10, after which $g$ returns to 0.2 forever

## Enter `Dolo.jl`

Let's perform these experiments with [Dolo.jl](https://github.com/EconForge/Dolo.jl).
This will allow us to cleanly separate the model definition from the code
used to exploit the results. General guidelines on how to write models are available
[here](http://www.econforge.org/Dolo.jl/latest/model_specification.html).

Here's the dolo version of the model that we will be using.

In [2]:
#Note: This assumes you are loading a file in the same directory as this .ipynb.
print(open(readstring, "rmt3_ch11.yaml"))

# Model in chapter 11 of Recursive Macroeconomic Theory 3rd edition by
# Ljvinquist and Sargent
name: fiscal_growth

symbols:
    states: [k, tau_c, tau_k]
    controls: [c]
    exogenous: [g, exog_tau_c, exog_tau_k]
    parameters: [beta, gamma, delta, alpha, A]

definitions:
    # Equations from 11.6.8
    eta: alpha*A*k^(alpha-1)
    w: A*k^alpha - k*eta
    Rb: (1+tau_c)/(1+tau_c(1))*((1-tau_k(1)) * (alpha*A*k(1)^(alpha-1) - delta) + 1.)

equations:

    arbitrage:
        # Equation 11.6.3
        - 1 = beta*(c(+1)/c)^(-gamma)*(1+tau_c)/(1+tau_c(1))*((1-tau_k(1))*(eta(1)-delta) + 1) | 0 <= c <= inf

    transition:
        # Equation 11.6.1
        - k = A*k(-1)^alpha+(1-delta)*k(-1)-c(-1)-g
        # We have the states tau_c and tau_k just follow exactly the sequence
        # of shocks that we supply.
        - tau_c = exog_tau_c
        - tau_k = exog_tau_k


calibration:
    # parameters
    beta: 0.95
    gamma: 2.0
    delta: 0.2
    alpha: 0.33
    A: 1.
    exog_tau_c: 0.0

The general formulation of a model in Dolo specifies a controlled process: 

$$s_t = g(m_{t-1}, s_{t-1}, x_{t-1}, m_tt)$$

where $s_t$ is a vector of states, $x_t$ a vector of controls taken at each state, and $m_t$ a vector of exogenous variables. This equation is defined in the `equations:transition` block.

For our model, there is essentially one state $k_t$ and one control $c_t$. Note that in this particular case choosing $c_t$ is equivalent to choosing $k_{t+1}$, but this is not true in general, for instance if there are many control for one state.

Notice in the model file the addition of `tau_c` and `tau_k` as state variables. These dummy states track the innovations `exog_tau_c`, `exog_tau_k`. This was necessary in order to to have `tau_c` in both period `t` and period `t+1` in the Euler equation defined in the block `equations:arbitrage` whose conventional definition is:

$$0 = E_t \left[ f(m_t, s_t, x_t, s_{t+1}, x_{t+1}, m_{t+1}) \right]$$

In this note we are interested in the predictible effect of a preannounced government policy on the decisions by the representative agent, not in a time-invariant decision rule. Hence we will use the `perfect_foresight` solution routine. This module implements the stacked time algorithm for solving determinsitc problems. A description of this algorithm can be found [here](http://www.econforge.org/Dolo.jl/latest/algos.html#Perfect-Foresight-1).

In a perfect foresight simulation, the computational cost of adding additional states is relatively low, because one does not need to solve for the optimal decisions at each point of the state space. For more information on which variables can appear at what times in each type of equation, see the [dolo model classification](http://www.econforge.org/Dolo.jl/latest/model_specification.html) . 

Let's now import the model and display it. The display shows the residuals of each equation, evaluated at the values specified in the `calibration` section of the modfile.

In [3]:
#using PlotlyJS
using Plots
gr()
using Dolo
using AxisArrays
model = yaml_import("rmt3_ch11.yaml")
model

name,fiscal_growth
filename,rmt3_ch11.yaml
Type,Equation
transition,\[k_{t} = \left(\left(A \left(k_{t-1}\right)^{\alpha}+\left(1-\delta\right) k_{t-1}-c_{t-1}\right)-g_{t}\right)\]
,"\[\tau_{c,t} = exog_{\tau,c,t}\]"
,"\[\tau_{k,t} = exog_{\tau,k,t}\]"
arbitrage,"\[1 = \frac{\beta \left(\frac{c_{t+1}}{c_{t}}\right)^{-\gamma} 1+\tau_{c,t}}{1+\tau_{c,t+1}} \left(1-\tau_{k,t+1}\right) \left(\eta_{t+1}-\delta\right)+1\]"


Now let's construct some dictionaries to hold the shocks in our experiments.

In the `deterministic_solve` function, simulations last for `T` periods. dolo assumes that if a given time-series of shocks is less than `T` in length, that the corresponding shock will hold its last given value until period `T`. Thus, to implement the once-and-for-all increase in our exogenous variables we simply need to pass the values before the increase and a single instance of the value after the increase. 

We do this below.

In [4]:
shocks_1 = Dict(:g => vcat(fill(0.2, 9), 0.4))
shocks_2 = Dict(:exog_tau_c => vcat(fill(0.0, 9), 0.2))
shocks_3 = Dict(:exog_tau_k => vcat(fill(0.0, 9), 0.2))
shocks_4 = Dict(:g => vcat(fill(0.2, 9), [0.4, 0.2]))
no_shocks = Dict(:g => [model.calibration.flat[:g]])

# also specify how long to simulate and plot
T = 101  # simulation length
p_T = 40;  # Periods to plot

## Experiment 1  (RMT 3 Figure 11.9.1)

Now we are ready to do the experiment corresponding to the once-and-for-all increase to $g$ in period $10$.

First, we solve for the equilibrium transition in this experiment

In [5]:
sol = perfect_foresight(model, shocks_1, T=T, complementarities=false);

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     1.973629e-02     2.192046e+00
     2     2.673832e-04     2.879876e-01
     3     1.064317e-07     5.991851e-03
     4     1.287859e-14     1.965924e-06


For comparison, we will also want to solve for the equilibrium path assuming no changes to any of the government variables ($g=0.2$, $\tau_c=0$, and $\tau_k=0$ forever)

In [6]:
sol_ref = perfect_foresight(model, no_shocks, T=T, complementarities=false)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------


2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:g, :exog_tau_c, :exog_tau_k, :k, :tau_c, :tau_k, :c, :w, :Rb, :eta]
    :T, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9  …  92, 93, 94, 95, 96, 97, 98, 99, 100, 101]
And data, a 10×102 Array{Float64,2}:
 0.2       0.2       0.2       0.2       …  0.2       0.2       0.2     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 1.48996   1.48996   1.48996   1.48996      1.48996   1.48996   1.48996 
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.642645  0.642645  0.642645  0.642645     0.642645  0.642645  0.642645
 0.764226  0.764226  0.764226  0.764226     0.764226  0.764226  0.764226
 1.05263   1.05263   1.05263   1.05263      1.05263   1.05263   1.05263 
 0.252632  0.252632  0.252632  0.252632     0.252632  0.252632  0.252632

We will also want to show the steady state. We could solve for it numerically,
but since the model is simple enough, the calibrated values given with 
the model, already solve it in closed form.

In [7]:
ss = model.calibration.flat[:k, :c, :eta, :w, :g, :Rb]
kbar, cbar, etabar, wbar, gbar, Rbar = ss;

Now all the computations are done, we will work on constructing figure 11.9.1 in RMT3.

Note that we will first construct the plot by hand, which will require some effort on our part. Then below we will show how to leverage convenience functions in `dolo` to make that task easier.

In [8]:
# Set up plotting materials 
variables = [:k, :c, :Rb, :eta, :g, :w]
#main_style = attr(marker_color=:"#E24A33")
#ss_style = attr(marker_color=:"black", line_dash=:"dash")
#other_style = attr(line_dash=:"dashdot", marker_color=:"#348ABD")
#my_sim_styles = [i.fields for i in (main_style, ss_style, other_style)]
marker_color = [:"#E24A33", :black, :"#348ABD"]
line_dash = [:line, :dash, :dashdot];

In [43]:
function plot_irfs(
        sims::Vector{<:AxisArray},
        variables::Vector{Symbol},
        #sim_styles::Vector{<:Associative}=[Dict() for i in 1:length(variables)],
        color_style::Array{Any},
        line_style::Array{Symbol},
        titles::Vector=variables,
        T::Int=length(sims[1][Axis{:T}]) 

    )
    n_row_col = Dict(
        1 => (1, 1),
        2 => (1, 2),
        3 => (1, 3),
        4 => (2, 2),
        5 => (2, 3),
        6 => (2, 3),
        7 => (4, 2),
        8 => (4, 2),
        9 => (3, 3),
        10 => (4, 3),
        11 => (4, 3),
        12 => (4, 3)
    )
    n_sim = length(sims)
    n_var = length(variables)
    
    if n_var > 12
        error("can only plot up to 12 variables for you...")
    end
    
### Method for PlotlyJS.jl
#    plots = map(1:n_var) do i_v
#        v = variables[i_v]
#        traces = GenericTrace[
#            PlotlyJS.scatter(sim_styles[i], y=Array(sims[i][v][1:T]), name=v) 
#            for i in 1:n_sim
#        ]
#        PlotlyJS.plot(traces, Layout(title=titles[i_v]))
#    end
        
#    nrow, ncol = n_row_col[n_var]
#    need_plots = nrow*ncol
#    append!(plots, fill(PlotlyJS.plot(), need_plots - n_var))
#    append!(plots, fill(Plots.plot(), need_plots - n_var))
    #rows = ntuple(i -> ncol, nrow)
    #relayout(hvcat(rows, plots...), showlegend=false)
    
####
    
    y = Array{Vector}(n_var)
    t = string.(titles)
    titles_string = Array{String}(1, n_var)
    
    
    for i_v in 1: n_var
        v = variables[i_v]
        y[i_v] = sims[1][v][1:T]
        titles_string[i_v] = t[i_v]
    end
    Plots.plot(y, layout = n_var, color=color_style[1], line= (line_style[1], 3), title = titles_string, legend=:none)
    if n_sim > 1
        if n_sim > 2
            for i_v in 1: n_var
                v = variables[i_v]
                y[i_v] = sims[2][v][1:T]
            end
            Plots.plot!(y, color=color_style[2], line=(line_style[2], 3), legend=:none)
            for i_v in 1: n_var
                v = variables[i_v]
                y[i_v] = sims[3][v][1:T]
            end
            Plots.plot!(y, color=color_style[3], line=(line_style[3], 3), legend=:none)
        else
    
            for i_v in 1: n_var
                v = variables[i_v]
                y[i_v] = sims[2][v][1:T]
            end
            Plots.plot!(y, color=color_style[2], line=(line_style[2], 3), legend=:none)
        end
        
    end                
        
end

plot_irfs (generic function with 3 methods)

In [47]:
variables = [:k, :c, :Rb, :eta, :g, :w]
fig_11_9_1 = plot_irfs([sol, sol_ref], variables, marker_color, line_dash, variables, p_T)
#relayout!(fig_11_9_1, title="RMT3 Figure 11.9.1")
#fig_11_9_1

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.5 
 
 
 1.6 
 
 
 1.7 
 
 
 1.8 
 
 
 1.9 
 
 
 2.0 
 
 
 k 
 
<polyline clip-path="url(#clip7902)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 38.8117,175.813 42.44,163.776 46.0683,150.763 49.6966,136.411 53.3249,120.347 56.9532,102.19 60.5815,81.5642 64.2098,58.1085 67.838,31.4961 71.4663,50.7908 
 75.0946,67.3264 78.7229,81.5355 82.3512,93.7753 85.9795,104.342 89.6078,113.483 93.2361,121.404 96.8643,128.279 100.493,134.255 104.121,139.457 107.749,143.989 
 111.377,147.943 115.006,151.394 118.634,154.41 122.262,157.046 125.891,159.353 129.519,161.373 133.147,163.141 136.776,164.691 140.404,166.05 144.032,167.241 
 147.66,168.285 151.289,169.202 154.917,170.006 158.545,170.712 162.174,171.332 165.802,171.876 169.43,172.354 173.058,172.774 176.687,173.143 180.315,173.466 
 
 "/>
<polyline clip-path="url(#clip7902)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 38.8117,175.813 42.44,175.813 46.0683,175.813 49.6966,175.813 53.3249,175.813 56.9532,175.813 60.5815,175.813 64.2098,175.813 67.838,175.813 71.4663,175.813 
 75.0946,175.813 78.7229,175.813 82.3512,175.813 85.9795,175.813 89.6078,175.813 93.2361,175.813 96.8643,175.813 100.493,175.813 104.121,175.813 107.749,175.813 
 111.377,175.813 115.006,175.813 118.634,175.813 122.262,175.813 125.891,175.813 129.519,175.813 133.147,175.813 136.776,175.813 140.404,175.813 144.032,175.813 
 147.66,175.813 151.289,175.813 154.917,175.813 158.545,175.813 162.174,175.813 165.802,175.813 169.43,175.813 173.058,175.813 176.687,175.813 180.315,175.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.45 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 c 
 
<polyline clip-path="url(#clip7903)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 238.812,66.9969 242.44,68.1058 246.068,70.3464 249.697,73.753 253.325,78.3647 256.953,84.2182 260.581,91.3392 264.21,99.7353 267.838,109.388 271.466,117.808 
 275.095,125.158 278.723,131.578 282.351,137.189 285.979,142.096 289.608,146.39 293.236,150.149 296.864,153.441 300.493,156.325 304.121,158.853 307.749,161.07 
 311.377,163.014 315.006,164.719 318.634,166.216 322.262,167.529 325.891,168.682 329.519,169.694 333.147,170.582 336.776,171.363 340.404,172.048 344.032,172.65 
 347.66,173.179 351.289,173.644 354.917,174.052 358.545,174.411 362.174,174.726 365.802,175.003 369.43,175.246 373.058,175.46 376.687,175.648 380.315,175.813 
 
 "/>
<polyline clip-path="url(#clip7903)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 238.812,31.4961 242.44,31.4961 246.068,31.4961 249.697,31.4961 253.325,31.4961 256.953,31.4961 260.581,31.4961 264.21,31.4961 267.838,31.4961 271.466,31.4961 
 275.095,31.4961 278.723,31.4961 282.351,31.4961 285.979,31.4961 289.608,31.4961 293.236,31.4961 296.864,31.4961 300.493,31.4961 304.121,31.4961 307.749,31.4961 
 311.377,31.4961 315.006,31.4961 318.634,31.4961 322.262,31.4961 325.891,31.4961 329.519,31.4961 333.147,31.4961 336.776,31.4961 340.404,31.4961 344.032,31.4961 
 347.66,31.4961 351.289,31.4961 354.917,31.4961 358.545,31.4961 362.174,31.4961 365.802,31.4961 369.43,31.4961 373.058,31.4961 376.687,31.4961 380.315,31.4961 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.01 
 
 
 1.02 
 
 
 1.03 
 
 
 1.04 
 
 
 1.05 
 
 
 Rb 
 
<polyline clip-path="url(#clip7904)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 438.812,46.9849 442.44,62.8336 446.068,79.3249 449.697,96.6685 453.325,114.993 456.953,134.34 460.581,154.659 464.21,175.813 467.838,160.666 471.466,146.873 
 475.095,134.366 478.723,123.065 482.351,112.888 485.979,103.748 489.608,95.5611 493.236,88.2442 496.864,81.7177 500

## Change in IES (RMT3 Figure 11.9.2)

We now want to highlight the impact of the parameter $\gamma$ in the household's preferences. The intertemporal elasticity of substitution (IES) for these preferences is given by $1/\gamma$. This means that higher values of $\gamma$ decrease IES, making the household less willing to substitute consumption across time.

Let's perform the same experiment as above, but this time with $\gamma$ set to $0.2$ instead of $2.0$.

In [45]:
# Change gamma and compute solution again
set_calibration!(model, gamma=0.2)
sol_ies = perfect_foresight(model, shocks_1, T=T, complementarities=false);

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     4.365597e-03     1.937257e+00
     2     2.997518e-05     4.629510e-02
     3     1.681675e-09     3.678048e-04


In [46]:
variables = [:k, :c, :Rb, :eta, :g, :w]
fig_11_9_2 = plot_irfs([sol, sol_ref, sol_ies], variables, marker_color, line_dash, variables, p_T)
#relayout!(fig_11_9_2, title="RMT3 Figure 11.9.2")
#fig_11_9_2

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.5 
 
 
 1.6 
 
 
 1.7 
 
 
 1.8 
 
 
 1.9 
 
 
 2.0 
 
 
 k 
 
<polyline clip-path="url(#clip7702)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 38.8117,175.813 42.44,163.776 46.0683,150.763 49.6966,136.411 53.3249,120.347 56.9532,102.19 60.5815,81.5642 64.2098,58.1085 67.838,31.4961 71.4663,50.7908 
 75.0946,67.3264 78.7229,81.5355 82.3512,93.7753 85.9795,104.342 89.6078,113.483 93.2361,121.404 96.8643,128.279 100.493,134.255 104.121,139.457 107.749,143.989 
 111.377,147.943 115.006,151.394 118.634,154.41 122.262,157.046 125.891,159.353 129.519,161.373 133.147,163.141 136.776,164.691 140.404,166.05 144.032,167.241 
 147.66,168.285 151.289,169.202 154.917,170.006 158.545,170.712 162.174,171.332 165.802,171.876 169.43,172.354 173.058,172.774 176.687,173.143 180.315,173.466 
 
 "/>
<polyline clip-path="url(#clip7702)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 38.8117,175.813 42.44,175.813 46.0683,175.813 49.6966,175.813 53.3249,175.813 56.9532,175.813 60.5815,175.813 64.2098,175.813 67.838,175.813 71.4663,175.813 
 75.0946,175.813 78.7229,175.813 82.3512,175.813 85.9795,175.813 89.6078,175.813 93.2361,175.813 96.8643,175.813 100.493,175.813 104.121,175.813 107.749,175.813 
 111.377,175.813 115.006,175.813 118.634,175.813 122.262,175.813 125.891,175.813 129.519,175.813 133.147,175.813 136.776,175.813 140.404,175.813 144.032,175.813 
 147.66,175.813 151.289,175.813 154.917,175.813 158.545,175.813 162.174,175.813 165.802,175.813 169.43,175.813 173.058,175.813 176.687,175.813 180.315,175.813 
 
 "/>
<polyline clip-path="url(#clip7702)" style="stroke:#348abd; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 6, 2, 6" points="
 38.8117,175.813 42.44,175.313 46.0683,174.614 49.6966,173.465 53.3249,171.454 56.9532,167.868 60.5815,161.471 64.2098,150.19 67.838,130.74 71.4663,147.787 
 75.0946,158.244 78.7229,164.743 82.3512,168.815 85.9795,171.38 89.6078,173.002 93.2361,174.028 96.8643,174.68 100.493,175.093 104.121,175.356 107.749,175.522 
 111.377,175.628 115.006,175.696 118.634,175.739 122.262,175.766 125.891,175.783 129.519,175.794 133.147,175.801 136.776,175.806 140.404,175.809 144.032,175.81 
 147.66,175.811 151.289,175.812 154.917,175.813 158.545,175.813 162.174,175.813 165.802,175.813 169.43,175.813 173.058,175.813 176.687,175.813 180.315,175.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.45 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 c 
 
<polyline clip-path="url(#clip7703)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 238.812,66.7043 242.44,67.804 246.068,70.0261 249.697,73.4046 253.325,77.9784 256.953,83.7835 260.581,90.8459 264.21,99.1727 267.838,108.746 271.466,117.097 
 275.095,124.386 278.723,130.753 282.351,136.318 285.979,141.185 289.608,145.443 293.236,149.171 296.864,152.436 300.493,155.296 304.121,157.804 307.749,160.002 
 311.377,161.93 315.006,163.621 318.634,165.105 322.262,166.407 325.891,167.551 329.519,168.554 333.147,169.436 336.776,170.21 340.404,170.89 344.032,171.487 
 347.66,172.011 351.289,172.472 354.917,172.877 358.545,173.233 362.174,173.545 365.802,173.82 369.43,174.061 373.058,174.274 376.687,174.46 380.315,174.624 
 
 "/>
<polyline clip-path="url(#clip7703)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 238.812,31.4961 242.44,31.4961 246.068,31.4961 249.697,31.4961 253.325,31.4961 256.953,31.4961 260.581,31.4961 264.21,31.4961 267.838,31.4961 271.466,31.4961 
 275.095,31.4961 278.723,31.4961 282.351,31.4961 285.979,31.4961 289.608,31.4961 293.236,31.4961 296.864,31.4961 300.493,31.4961 304.121,31.4961 307.749,31.4961 
 311.377,31.4961 315.006,31.4961 318.634,31.4961 322.262,31.4961 325.891,31.4961 

In the figure above the solid red line represents the experiment when $\gamma=2.0$, the dash-dotted blue line tracks the equilibrium in the experiment when $\gamma=0.2$, and the black dashed line tracks the steady state.

Notice that because the household is more willing to move consumption across time when $\gamma$ is smaller, the movement from initial to final levels of consumption is both larger and quicker. This lets the consumption stream "swallow" most of the impact of the increase in $g$ and capital doesn't respond as much (think about the household budget constraint). Factor prices and $\bar{R}$ are functions of the captial stock, so they also do not respond as sharply.

## Consumption tax shocks (RMT 3 Figure 11.9.4)

Now let's consider the second experiment: a once-and-for-all increase in $\tau_c$ from 0 to 0.2 in period 10.

In [40]:
# reset gamma
set_calibration!(model, gamma=2.0)

sol2 = perfect_foresight(model, shocks_2, T=T, complementarities=false)

set_calibration!(model, gamma=0.2)

sol2_ies = perfect_foresight(model, shocks_2, T=T, complementarities=false)

variables=[:k, :c, :Rb, :eta, :tau_c]
fig_11_9_4 = plot_irfs([sol2, sol_ref, sol2_ies], variables, marker_color, line_dash, variables, p_T)
#relayout!(fig_11_9_4, title="RMT4 figure 11.9.4")
#fig_11_9_4

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     2.048037e-02     1.965409e+00
     2     2.372421e-04     4.325968e-02
     3     3.251771e-08     5.450922e-04
     4     8.881784e-16     7.802051e-08
Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     4.976031e-02     2.096298e+00
     2     3.567140e-03     1.512156e-01
     3     1.885802e-05     1.069803e-02
     4     5.363958e-10     5.290224e-05


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.1 
 
 
 1.2 
 
 
 1.3 
 
 
 1.4 
 
 
 k 
 
<polyline clip-path="url(#clip7302)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 38.8117,31.4961 42.44,34.3514 46.0683,37.4591 49.6966,40.9464 53.3249,44.9634 56.9532,49.6916 60.5815,55.3546 64.2098,62.233 67.838,70.6844 71.4663,81.173 
 75.0946,74.0079 78.7229,67.8545 82.3512,62.5759 85.9795,58.0523 89.6078,54.1789 93.2361,50.8645 96.8643,48.03 100.493,45.6072 104.121,43.5372 107.749,41.7692 
 111.377,40.2596 115.006,38.9709 118.634,37.8712 122.262,36.9327 125.891,36.1321 129.519,35.4491 133.147,34.8666 136.776,34.3698 140.404,33.9461 144.032,33.5849 
 147.66,33.2768 151.289,33.0141 154.917,32.7902 158.545,32.5993 162.174,32.4365 165.802,32.2977 169.43,32.1794 173.058,32.0786 176.687,31.9926 180.315,31.9193 
 
 "/>
<polyline clip-path="url(#clip7302)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 38.8117,31.4961 42.44,31.4961 46.0683,31.4961 49.6966,31.4961 53.3249,31.4961 56.9532,31.4961 60.5815,31.4961 64.2098,31.4961 67.838,31.4961 71.4663,31.4961 
 75.0946,31.4961 78.7229,31.4961 82.3512,31.4961 85.9795,31.4961 89.6078,31.4961 93.2361,31.4961 96.8643,31.4961 100.493,31.4961 104.121,31.4961 107.749,31.4961 
 111.377,31.4961 115.006,31.4961 118.634,31.4961 122.262,31.4961 125.891,31.4961 129.519,31.4961 133.147,31.4961 136.776,31.4961 140.404,31.4961 144.032,31.4961 
 147.66,31.4961 151.289,31.4961 154.917,31.4961 158.545,31.4961 162.174,31.4961 165.802,31.4961 169.43,31.4961 173.058,31.4961 176.687,31.4961 180.315,31.4961 
 
 "/>
<polyline clip-path="url(#clip7302)" style="stroke:#348abd; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 6, 2, 6" points="
 38.8117,31.4961 42.44,32.1808 46.0683,33.1403 49.6966,34.7253 53.3249,37.5305 56.9532,42.634 60.5815,52.0667 64.2098,69.8199 67.838,104.306 71.4663,175.813 
 75.0946,119.487 78.7229,83.9337 82.3512,62.3403 85.9795,49.5033 89.6078,41.9636 93.2361,37.5657 96.8643,35.0105 100.493,33.5293 104.121,32.6718 107.749,32.1758 
 111.377,31.8889 115.006,31.7231 118.634,31.6273 122.262,31.5719 125.891,31.5399 129.519,31.5214 133.147,31.5107 136.776,31.5045 140.404,31.501 144.032,31.4989 
 147.66,31.4977 151.289,31.497 154.917,31.4966 158.545,31.4964 162.174,31.4962 165.802,31.4962 169.43,31.4961 173.058,31.4961 176.687,31.4961 180.315,31.4961 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 c 
 
<polyline clip-path="url(#clip7303)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 238.812,99.9241 242.44,99.8162 246.068,99.5897 249.697,99.2284 253.325,98.7095 256.953,98.0016 260.581,97.0622 264.21,95.8338 267.838,94.2378 271.466,113.917 
 275.095,112.304 278.723,110.929 282.351,109.756 285.979,108.757 289.608,107.905 293.236,107.179 296.864,106.559 300.493,106.031 304.121,105.582 307.749,105.198 
 311.377,104.871 315.006,104.592 318.634,104.355 322.262,104.152 325.891,103.98 329.519,103.833 333.147,103.707 336.776,103.6 340.404,103.509 344.032,103.431 
 347.66,103.365 351.289,103.309 354.917,103.261 358.545,103.22 362.174,103.185 365.802,103.155 369.43,103.13 373.058,103.108 376.687,103.089 380.315,103.074 
 
 "/>
<polyline clip-path="url(#clip7303)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 238.812,102.983 242.44,102.983 246.068,102.983 249.697,102.983 253.325,102.983 256.953,102.983 260.581,102.983 264.21,102.983 267.838,102.983 271.466,102.983 
 275.095,102.983 278.723,102.983 282.351,102.983 285.979,102.983 289.608,102.983 293.236,102.983 296.864,102.983 300.493,102.983 304.121,102.983 307.749,102.983 
 311.377,102.983 315.006,102.983 318.634,102.983 322.262,102.983 325.891,102.983 329.519,102.983 333.147,102.98

## Capital tax shocks (RMT 3 Figure 11.9.5)

Next, we turn to the third experiment: a once-and-for-all increase in $\tau_k$ from 0 to 0.2 in period 10

In [48]:
sol3 = perfect_foresight(model, shocks_3, T=T, complementarities=false)

# here we also want to look at the impact of changing the ies
set_calibration!(model, gamma=0.2)
sol3_ies = perfect_foresight(model, shocks_3, T=T, complementarities=false)

set_calibration!(model, gamma=2.0)  # reset gamma

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     1.560191e-03     2.123515e+00
     2     1.677118e-05     1.551257e-01
     3     1.569939e-09     1.471872e-03
Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     1.560191e-03     2.123515e+00
     2     1.677118e-05     1.551257e-01
     3     1.569939e-09     1.471872e-03


Dolo.ModelCalibration(Dolo.FlatCalibration(:exog_tau_c=>0.0,:A=>1.0,:exog_tau_k=>0.0,:gamma=>2.0,:delta=>0.2,:g=>0.2,:beta=>0.95,:tau_c=>0.0,:alpha=>0.33,:tau_k=>0.0…), Dolo.GroupedCalibration(:controls=>[0.642645],:states=>[1.48996, 0.0, 0.0],:exogenous=>[0.2, 0.0, 0.0],:parameters=>[0.95, 2.0, 0.2, 0.33, 1.0]), Dict(:alpha=>(:parameters, 4),:exog_tau_c=>(:exogenous, 2),:gamma=>(:parameters, 2),:c=>(:controls, 1),:delta=>(:parameters, 3),:exog_tau_k=>(:exogenous, 3),:tau_k=>(:states, 3),:A=>(:parameters, 5),:k=>(:states, 1),:g=>(:exogenous, 1)…), DataStructures.OrderedDict(:controls=>Symbol[:c],:states=>Symbol[:k, :tau_c, :tau_k],:exogenous=>Symbol[:g, :exog_tau_c, :exog_tau_k],:parameters=>Symbol[:beta, :gamma, :delta, :alpha, :A]))

In [50]:
variables = [:k, :c, :Rb, :eta, :tau_k]
fig_11_9_5 = plot_irfs([sol3, sol_ref, sol3_ies], variables, marker_color, line_dash, variables, p_T)
#relayout!(fig_11_9_5, title="RMT4 Figure 11.9.5")
#fig_11_9_5

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.40 
 
 
 1.42 
 
 
 1.44 
 
 
 1.46 
 
 
 1.48 
 
 
 k 
 
<polyline clip-path="url(#clip8302)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 51.991,31.4961 55.4503,31.9688 58.9096,32.6306 62.369,33.7212 65.8283,35.643 69.2876,39.1111 72.7469,45.4261 76.2063,56.9833 79.6656,78.2546 83.1249,117.772 
 86.5842,141.179 90.0435,155.11 93.5029,163.424 96.9622,168.394 100.422,171.369 103.881,173.15 107.34,174.218 110.799,174.857 114.259,175.24 117.718,175.47 
 121.177,175.608 124.637,175.69 128.096,175.739 131.555,175.769 135.015,175.787 138.474,175.798 141.933,175.804 145.393,175.808 148.852,175.81 152.311,175.811 
 155.771,175.812 159.23,175.813 162.689,175.813 166.149,175.813 169.608,175.813 173.067,175.813 176.527,175.813 179.986,175.813 183.445,175.813 186.905,175.813 
 
 "/>
<polyline clip-path="url(#clip8302)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 51.991,31.4961 55.4503,31.4961 58.9096,31.4961 62.369,31.4961 65.8283,31.4961 69.2876,31.4961 72.7469,31.4961 76.2063,31.4961 79.6656,31.4961 83.1249,31.4961 
 86.5842,31.4961 90.0435,31.4961 93.5029,31.4961 96.9622,31.4961 100.422,31.4961 103.881,31.4961 107.34,31.4961 110.799,31.4961 114.259,31.4961 117.718,31.4961 
 121.177,31.4961 124.637,31.4961 128.096,31.4961 131.555,31.4961 135.015,31.4961 138.474,31.4961 141.933,31.4961 145.393,31.4961 148.852,31.4961 152.311,31.4961 
 155.771,31.4961 159.23,31.4961 162.689,31.4961 166.149,31.4961 169.608,31.4961 173.067,31.4961 176.527,31.4961 179.986,31.4961 183.445,31.4961 186.905,31.4961 
 
 "/>
<polyline clip-path="url(#clip8302)" style="stroke:#348abd; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 6, 2, 6" points="
 51.991,31.4961 55.4503,31.9688 58.9096,32.6306 62.369,33.7212 65.8283,35.643 69.2876,39.1111 72.7469,45.4261 76.2063,56.9833 79.6656,78.2546 83.1249,117.772 
 86.5842,141.179 90.0435,155.11 93.5029,163.424 96.9622,168.394 100.422,171.369 103.881,173.15 107.34,174.218 110.799,174.857 114.259,175.24 117.718,175.47 
 121.177,175.608 124.637,175.69 128.096,175.739 131.555,175.769 135.015,175.787 138.474,175.798 141.933,175.804 145.393,175.808 148.852,175.81 152.311,175.811 
 155.771,175.812 159.23,175.813 162.689,175.813 166.149,175.813 169.608,175.813 173.067,175.813 176.527,175.813 179.986,175.813 183.445,175.813 186.905,175.813 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.64 
 
 
 0.65 
 
 
 0.66 
 
 
 0.67 
 
 
 c 
 
<polyline clip-path="url(#clip8303)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 245.401,147.266 248.861,146.745 252.32,145.495 255.779,143.041 259.239,138.457 262.698,130.009 266.157,114.45 269.617,85.6223 273.076,31.4961 276.535,89.9432 
 279.995,124.57 283.454,145.179 286.913,157.48 290.373,164.835 293.832,169.237 297.291,171.873 300.751,173.452 304.21,174.398 307.669,174.965 311.129,175.305 
 314.588,175.509 318.047,175.631 321.506,175.704 324.966,175.748 328.425,175.774 331.884,175.79 335.344,175.799 338.803,175.805 342.262,175.808 345.722,175.81 
 349.181,175.812 352.64,175.812 356.1,175.813 359.559,175.813 363.018,175.813 366.478,175.813 369.937,175.813 373.396,175.813 376.856,175.813 380.315,175.813 
 
 "/>
<polyline clip-path="url(#clip8303)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 245.401,148.766 248.861,148.766 252.32,148.766 255.779,148.766 259.239,148.766 262.698,148.766 266.157,148.766 269.617,148.766 273.076,148.766 276.535,148.766 
 279.995,148.766 283.454,148.766 286.913,148.766 290.373,148.766 293.832,148.766 297.291,148.766 300.751,148.766 304.21,148.766 307.669,148.766 311.129,148.766 
 314.588,148.766 318.047,148.766 321.506,148.766 324.966,148.766 328.425,148.766 331.884,148.

## Impulse shock to g (RMT 3 Figure 11.9.6)

Finally, we turn to the fourth experiment: a one-time shock to $g$ from 0.2 to 0.4 in period 10, then back to 0.2 forever.

In [49]:
sol4 = perfect_foresight(model, shocks_4, T=T, complementarities=false)

variables = [:k, :c, :Rb, :eta, :g]
fig_11_9_6 = plot_irfs([sol4, sol_ref], variables, marker_color, line_dash, variables, p_T)
#relayout!(fig_11_9_6, title="RMT4 Figure 11.9.6")
#fig_11_9_6

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     7.481441e-04     2.196956e-01
     2     3.695996e-07     4.228965e-03
     3     7.527312e-14     1.829582e-06


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.425 
 
 
 1.450 
 
 
 1.475 
 
 
 1.500 
 
 
 1.525 
 
 
 1.550 
 
 
 1.575 
 
 
 k 
 
<polyline clip-path="url(#clip8102)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 45.4014,110.126 48.8607,104.149 52.32,97.6558 55.7793,90.4054 59.2387,82.1227 62.698,72.4905 66.1573,61.1403 69.6166,47.6423 73.0759,31.4961 76.5353,175.813 
 79.9946,166.238 83.4539,158.043 86.9132,151.034 90.3726,145.042 93.8319,139.921 97.2912,135.547 100.751,131.813 104.21,128.624 107.669,125.903 111.129,123.581 
 114.588,121.6 118.047,119.909 121.506,118.468 124.966,117.238 128.425,116.19 131.884,115.296 135.344,114.533 138.803,113.883 142.262,113.329 145.722,112.856 
 149.181,112.453 152.64,112.11 156.1,111.817 159.559,111.568 163.018,111.355 166.478,111.173 169.937,111.019 173.396,110.887 176.856,110.775 180.315,110.679 
 
 "/>
<polyline clip-path="url(#clip8102)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 45.4014,110.126 48.8607,110.126 52.32,110.126 55.7793,110.126 59.2387,110.126 62.698,110.126 66.1573,110.126 69.6166,110.126 73.0759,110.126 76.5353,110.126 
 79.9946,110.126 83.4539,110.126 86.9132,110.126 90.3726,110.126 93.8319,110.126 97.2912,110.126 100.751,110.126 104.21,110.126 107.669,110.126 111.129,110.126 
 114.588,110.126 118.047,110.126 121.506,110.126 124.966,110.126 128.425,110.126 131.884,110.126 135.344,110.126 138.803,110.126 142.262,110.126 145.722,110.126 
 149.181,110.126 152.64,110.126 156.1,110.126 159.559,110.126 163.018,110.126 166.478,110.126 169.937,110.126 173.396,110.126 176.856,110.126 180.315,110.126 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.625 
 
 
 0.630 
 
 
 0.635 
 
 
 0.640 
 
 
 c 
 
<polyline clip-path="url(#clip8103)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 245.401,86.6093 248.861,88.4914 252.32,92.4003 255.779,98.5482 259.239,107.221 262.698,118.784 266.157,133.692 269.617,152.488 273.076,175.813 276.535,154.519 
 279.995,136.365 283.454,120.889 286.913,107.696 290.373,96.4503 293.832,86.8637 297.291,78.6918 300.751,71.7258 304.21,65.7879 307.669,60.7264 311.129,56.4119 
 314.588,52.7342 318.047,49.5993 321.506,46.9271 324.966,44.6494 328.425,42.7078 331.884,41.0528 335.344,39.6422 338.803,38.4397 342.262,37.4147 345.722,36.5411 
 349.181,35.7964 352.64,35.1616 356.1,34.6205 359.559,34.1593 363.018,33.7662 366.478,33.4311 369.937,33.1454 373.396,32.902 376.856,32.6944 380.315,32.5175 
 
 "/>
<polyline clip-path="url(#clip8103)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 245.401,31.4961 248.861,31.4961 252.32,31.4961 255.779,31.4961 259.239,31.4961 262.698,31.4961 266.157,31.4961 269.617,31.4961 273.076,31.4961 276.535,31.4961 
 279.995,31.4961 283.454,31.4961 286.913,31.4961 290.373,31.4961 293.832,31.4961 297.291,31.4961 300.751,31.4961 304.21,31.4961 307.669,31.4961 311.129,31.4961 
 314.588,31.4961 318.047,31.4961 321.506,31.4961 324.966,31.4961 328.425,31.4961 331.884,31.4961 335.344,31.4961 338.803,31.4961 342.262,31.4961 345.722,31.4961 
 349.181,31.4961 352.64,31.4961 356.1,31.4961 359.559,31.4961 363.018,31.4961 366.478,31.4961 369.937,31.4961 373.396,31.4961 376.856,31.4961 380.315,31.4961 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.045 
 
 
 1.050 
 
 
 1.055 
 
 
 1.060 
 
 
 Rb 
 
<polyline clip-path="url(#clip8104)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 445.401,106.709 448.861,113.161 452.32,120.299 455.779,128.369 459.239,137.641 462.698,148.416 466.157,161.022 469.617,175.813 473.076,31.4961 476.535,41.9923 
 479.995,50.8615 483.454,58.3658 486.913,64.7222 490.373,70.1114 493.832,74.6842 497.291,78.567 